In [ ]:
import os
import requests
import pandas as pd
import pickle
from requests.auth import HTTPBasicAuth


In [ ]:
# CONFIGURACION DEL AGENTE INTELIGENTE

SERVICENOW_INSTANCE = os.getenv("SERVICENOW_INSTANCE")
USERNAME = os.getenv("SERVICENOW_USERNAME")
PASSWORD = os.getenv("SERVICENOW_PASSWORD")
QUEUE_NAME = os.getenv("SERVICENOW_QUEUE")

TABLE_API = f"{SERVICENOW_INSTANCE}/api/now/table/incident"

DATABRICKS_INSTANCE = os.getenv("DATABRICKS_INSTANCE")
DATABRICKS_TOKEN = os.getenv("DATABRICKS_TOKEN")
DATABRICKS_HEADERS = {
    "Authorization": f"Bearer {DATABRICKS_TOKEN}",
    "Content-Type": "application/json"
}



In [ ]:
# FUNCIÓN PARA DESCARGAR LOS TICKETS DE SERVICE NOW

def get_tickets(queue_name):
    params = {
        "sysparm_query": f"assignment_group={queue_name}",
        "sysparm_limit": "10000",
        "sysparm_display_value": "true"
    }

    response = requests.get(
        TABLE_API,
        auth=HTTPBasicAuth(USERNAME, PASSWORD),
        headers={"Accept": "application/json"},
        params=params
    )

    if response.status_code != 200:
        raise Exception(f"Error al conectar a ServiceNow: {response.status_code}, {response.text}")

    data = response.json()
    tickets = data.get("result", [])
    return pd.DataFrame(tickets)


In [ ]:
# LECTURA DEL MODELO BERT CON TOKENS PERSONALIZADOS PARA ASIGNAR LOS TICKETS

def classify_tickets(df, model_path="label_encoder.pkl"):
    with open(model_path, "rb") as f:
        label_encoder = pickle.load(f)

    if "short_description" not in df.columns:
        raise Exception("No se encontró la columna 'short_description' en los tickets")

    df["suggested_agent"] = label_encoder.transform(df["short_description"])
    return df


In [ ]:
# HUMAN IN THE LOOP

def human_in_the_loop(df):
    validated_agents = []
    print("\n--- VALIDACIÓN HUMANA ---")
    for idx, row in df.iterrows():
        print(f"\nTicket ID: {row.get('sys_id', 'N/A')}")
        print(f"Descripción: {row['short_description']}")
        print(f"Agente sugerido: {row['suggested_agent']}")

        user_input = input("¿Aceptar asignación? (Enter para aceptar, o escribe nuevo agente): ")
        if user_input.strip() == "":
            validated_agents.append(row["suggested_agent"])
        else:
            validated_agents.append(user_input.strip())

    df["validated_agent"] = validated_agents
    return df



In [ ]:
# DAR ACCESO CUANDO ES SOLO LECTURA A TABLAS NO SENSIBLES

def grant_databricks_table_readonly(user_email, catalog="brewdat_uc_maz_prod", schema="default", tables=None):
    """
    Otorga acceso de solo lectura a tablas específicas dentro de un catálogo y esquema en Unity Catalog.
    :param user_email: Email del usuario
    :param catalog: Nombre del catálogo
    :param schema: Nombre del esquema
    :param tables: Lista de tablas a las que dar acceso
    """
    if tables is None:
        tables = []

    for table in tables:
        endpoint = f"{DATABRICKS_INSTANCE}/api/2.1/unity-catalog/permissions/table/{catalog}.{schema}.{table}"
        payload = {
            "access_control_list": [
                {
                    "principal": user_email,
                    "permissions": ["SELECT"]  # solo lectura
                }
            ]
        }

        response = requests.patch(endpoint, headers=DATABRICKS_HEADERS, json=payload)

        if response.status_code == 200:
            print(f"✅ Acceso de solo lectura otorgado a tabla '{catalog}.{schema}.{table}' para {user_email}")
        else:
            print(f"⚠️ Error al otorgar acceso a {table}: {response.status_code}, {response.text}")



In [ ]:
if __name__ == "__main__":
    print("Descargando tickets desde ServiceNow...")
    tickets_df = get_tickets(QUEUE_NAME)

    print("Guardando tickets crudos en Excel...")
    tickets_df.to_excel("tickets_raw.xlsx", index=False)

    print("Clasificando tickets con modelo entrenado...")
    classified_df = classify_tickets(tickets_df)

    print("Iniciando validación humana...")
    validated_df = human_in_the_loop(classified_df)

    print("\n--- PROCESANDO ACCESOS DATABRICKS ---")
    for idx, row in validated_df.iterrows():
        if "ACCESO DATABRICKS" in row["short_description"].upper():
            user_email = row.get("caller_id", None)
            if user_email:
                grant_databricks_readonly_access(user_email)
            else:
                print(f"⚠️ Ticket {row.get('sys_id')} no tiene email de usuario")

    print("Guardando tickets validados en Excel...")
    validated_df.to_excel("tickets_classified.xlsx", index=False)

    print("Proceso completado con Human-in-the-Loop y acceso Databricks ✅")
